In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
from google.colab import drive
drive._mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/코딩관련/colab/nlp/bert_error/

[Errno 2] No such file or directory: 'drive/MyDrive/코딩관련/colab/nlp/bert_error/'
/content/drive/MyDrive/코딩관련/colab/nlp/bert_error


In [ ]:
!pip install gluonnlp==0.10.0 pandas tqdm   #0.6.0, 0.8.0
!pip install mxnet
 
# !pip install sentencepiece
#!pip install transformers==3.0.2
# !pip install torch==1.2.0
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.2 

ERROR: Could not find a version that satisfies the requirement torch==1.8.2 (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2, 0.4.1, 0.4.1.post2, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0)
ERROR: No matching distribution found for torch==1.8.2


In [ ]:
# SKT KoBERT_hf
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-wz9k3ylm/kobert-tokenizer_29a3083b1b9c43179f51ae8fda39085e
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-wz9k3ylm/kobert-tokenizer_29a3083b1b9c43179f51ae8fda39085e


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#kobert
#from kobert.utils import get_tokenizer
from kobert_tokenizer import KoBERTTokenizer
#from kobert.pytorch_kobert import get_pytorch_kobert_model




#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")


#device = torch.device("cpu")

In [ ]:
# SKT KoBERT
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xbh2ihbh
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xbh2ihbh


In [ ]:
#from kobert.pytorch_kobert import get_kobert_model
#tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
#bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

In [ ]:
#BERT 모델, Vocabulary 불러오기
#bertmodel, vocab = get_pytorch_kobert_model()

#bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')

In [ ]:
#bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

In [ ]:
# 챗봇 데이터셋
chatbot_data = pd.read_excel('data.xlsx')

In [ ]:

# answer json 파일 열기
import json
from google.colab import files
with open('answer.json') as json_file:
   answer_data = json.load(json_file)



In [ ]:
#chatbot_data.sample(n=10)

In [ ]:
#print(answer_data)

In [ ]:
# 특수문자 제거하기
import re
def cleanText(input):
  clean = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',input)
  return clean

In [ ]:
# 네이버 맞춤법 교정기 기반 라이브러리
!pip install git+https://github.com/ssut/py-hanspell.git

from hanspell import spell_checker

def spellCheck(input):
  checked = spell_checker.check(input)
  #print(checked.as_dict())
  #print(checked.as_dict()['checked'])
  return checked.as_dict()['checked']


  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-lmlrdvkw
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-lmlrdvkw


In [ ]:
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_방법"), '클래스'] = 0  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_시기"), '클래스'] = 1  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_이후"), '클래스'] = 2  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_화분"), '클래스'] = 3  
chatbot_data.loc[(chatbot_data['클래스'] == "분갈이_흙"), '클래스'] = 4  
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_화상"), '클래스'] = 5 
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_온도"), '클래스'] = 6
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_인조조명"), '클래스'] = 7
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_장소"), '클래스'] = 8
chatbot_data.loc[(chatbot_data['클래스'] == "재배환경_햇빛"), '클래스'] = 9
chatbot_data.loc[(chatbot_data['클래스'] == "튜토리얼_방법"), '클래스'] = 10
chatbot_data.loc[(chatbot_data['클래스'] == "물주기"), '클래스'] = 11 
chatbot_data.loc[(chatbot_data['클래스'] == "물주기_저면관수"), '클래스'] = 12 
chatbot_data.loc[(chatbot_data['클래스'] == "물주기_계절"), '클래스'] = 13
chatbot_data.loc[(chatbot_data['클래스'] == "번식_잎꽂이방법"), '클래스'] = 14 
chatbot_data.loc[(chatbot_data['클래스'] == "번식_자구"), '클래스'] = 15
chatbot_data.loc[(chatbot_data['클래스'] == "번식_적심"), '클래스'] = 16
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_무름병"), '클래스'] = 17
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_썩음병"), '클래스'] = 18
chatbot_data.loc[(chatbot_data['클래스'] == "뿌리줄기_기타"), '클래스'] = 19
chatbot_data.loc[(chatbot_data['클래스'] == "웃자람_증상"), '클래스'] = 20
chatbot_data.loc[(chatbot_data['클래스'] == "웃자람_해결"), '클래스'] = 21 
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_곰팡이"), '클래스'] = 22
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_과습"), '클래스'] = 23
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_과습여부"), '클래스'] = 24
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_마름"), '클래스'] = 25
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_원인알수없음"), '클래스'] = 26
chatbot_data.loc[(chatbot_data['클래스'] == "잎의질병_탄저병"), '클래스'] = 27
chatbot_data.loc[(chatbot_data['클래스'] == "해충_기타"), '클래스'] = 28
chatbot_data.loc[(chatbot_data['클래스'] == "해충_깍지"), '클래스'] = 29
chatbot_data.loc[(chatbot_data['클래스'] == "해충_응애"), '클래스'] = 30
chatbot_data.loc[(chatbot_data['클래스'] == "해충_해충약"), '클래스'] = 31


data_list = []
for ques, label in zip(chatbot_data['질문내용'], chatbot_data['클래스'])  :
    data = []
    ques = cleanText(ques)
    #ques = spellCheck(ques)
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[5])
print(data_list[123])
print(data_list[200])

['회색곰팡이 같은데 조언을 구합니다', '22']
['잎이 물렁물렁해', '23']
['무름병인지 아닌지 모르겠네요', '24']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

2228
558


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10  #원래15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
!pip install kobert_transformers==0.4.1

In [ ]:
#다른 버트 파일 임포트하는 방안- vocab 다운시 오류, 대신 get_tokenizer().vocab_file 쓰면 BERTDataset에서 오류 
from kobert_transformers import get_kobert_model
from kobert_transformers import get_tokenizer
#bertmodel,vocab = get_kobert_model()
 

In [ ]:
#skt 허깅페이스 
from kobert_transformers import get_tokenizer
from kobert.pytorch_kobert import get_kobert_model
 
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
 
#print(tokenizer)
#print(vocab)

In [ ]:
#connection error
#from kobert.pytorch_kobert import get_pytorch_kobert_model
#a,b=get_pytorch_kobert_model()

In [ ]:
#vocab2=KoBERTTokenizer.get_vocab(tokenizer)
#KoBERTTokenizer

In [ ]:
#토큰화
tok=tokenizer.tokenize
 
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False) #이 코드가 tok("날씨 좋아") 했을때 먹히도록..
#print("현상태:",tok.basic_tokenizer("오늘은 날씨가 좋아."))
#nlp.data.BERTSentenceTransform
#tok.sentencepiece
sample="오늘은 날씨가 좋아."
print("정답:", tok(sample))
#print(tok(sample))
 
transform = nlp.data.BERTSentenceTransform(tokenizer=tok,
                                           vocab=vocab,
                                           max_seq_length=140,
                                           pad=True,
                                           pair=False)
from_string = transform(['오늘은 날씨가 좋아.'])
sentences = transform([sample])
print(sentences)
#tok2=tokenizer.tokenize()

정답: ['▁오늘', '은', '▁날씨', '가', '▁좋아', '.']
(array([   2, 3419, 7086, 1408, 5330, 4207,   54,    3,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), array(8, dty

In [ ]:
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok,vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=32,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
from transformers import BertModel
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
#model = BertModel.from_pretrained('skt/kobert-base-v1').to(device)
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
#device = torch.device("cpu")

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/35 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.530857563018799 train acc 0.03125
epoch 1 train acc 0.04038461538461539


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.8820199275362319


  0%|          | 0/35 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.4912557005882263 train acc 0.984375
epoch 10 train acc 0.9698832417582417


  0%|          | 0/9 [00:00<?, ?it/s]

epoch 10 test acc 0.883756038647343


In [ ]:
import random 
#index가 before idx 
def getAnswer(index, num):
  #answers=[]
  for tg in answer_data["intents"]:
    if tg['tag']==str(index):
      if num == 0:
        answers = tg['answer']
      elif num==1:
        answers = tg['answeryes']
      elif num==2:
        answers = tg['answerno']


  #ans = random.choice(answers)
  ans=answers[0]
  return ans

In [ ]:
def predict(predict_sentence,beforeidx):

    predict_sentence = spellCheck(predict_sentence)
    predict_sentence = cleanText(predict_sentence)

    if predict_sentence=='응' and (beforeidx==3 or beforeidx==15 or beforeidx==17 or beforeidx==18 or beforeidx==22 or beforeidx==25 or beforeidx==28 or beforeidx==31):
        answer = getAnswer(beforeidx,1)
        return answer,-1
    elif predict_sentence=='아니' and (beforeidx==3 or beforeidx==15 or beforeidx==17 or beforeidx==18 or beforeidx==22 or beforeidx==25 or beforeidx==28 or beforeidx==31):
        answer = getAnswer(beforeidx,2)
        return answer,-1
    else:
        #print("전처리 후: "+predict_sentence)
    
        data = [predict_sentence, '0']
        #print(data) #['물 줄까', '0']

        dataset_another = [data]
        #print(dataset_another) #[['물 줄까', '0']]

        another_test = BERTDataset(dataset_another, 0, 1, tok,vocab, max_len, True, False)
    
        test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2) 
    
        model.eval()

        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)

            valid_length= valid_length
            label = label.long().to(device)

        
        
            out = model(token_ids, valid_length, segment_ids)
            tmp = out.detach().cpu().numpy()
            maxProb = np.max(tmp[0])
            test_eval=[]
            idx = -1
            for i in out:
                logits=i
                logits = logits.detach().cpu().numpy()

                idx = np.argmax(logits)

                if idx == 0:
                    test_eval.append("분갈이_방법")
                elif idx == 1:
                    test_eval.append("분갈이_시기")
                    
                elif idx == 2:
                    test_eval.append("분갈이_이후")
                    
                elif idx == 3:
                    test_eval.append("분갈이_화분")
                    beforeidx=idx
                elif idx == 4:
                    test_eval.append("분갈이_흙")
                elif idx == 5:
                    test_eval.append("잎의질병_화상")
                elif idx == 6:
                    test_eval.append( "재배환경_온도" )
                elif idx == 7:
                    test_eval.append( "재배환경_인조조명"  )
                elif idx == 8:
                    test_eval.append( "재배환경_장소")
                elif idx == 9:
                    test_eval.append("재배환경_햇빛" )
                elif idx == 10:
                    test_eval.append("튜토리얼_방법" )
                elif idx == 11:
                    test_eval.append("물주기" )
                elif idx == 12:
                    test_eval.append("물주기_저면관수" )
                elif idx == 13:
                    test_eval.append("물주기_계절" )
                    beforeidx=idx
                elif idx == 14:
                    test_eval.append( "번식_잎꽂이방법" )
                elif idx == 15:
                    test_eval.append("번식_자구" )
                    beforeidx=idx
                elif idx == 16:
                    test_eval.append(  "번식_적심")
                elif idx == 17:
                    test_eval.append("뿌리줄기_무름병" )
                    beforeidx=idx
                elif idx == 18:
                    test_eval.append("뿌리줄기_썩음병" )
                    beforeidx=idx
                elif idx == 19:
                    test_eval.append( "뿌리줄기_기타"  )
                elif idx == 20:
                    test_eval.append( "웃자람_증상"  )
                elif idx == 21:
                    test_eval.append("웃자람_해결"  )
                elif idx == 22:
                    test_eval.append( "잎의질병_곰팡이" )
                    beforeidx=idx
                elif idx == 23:
                    test_eval.append( "잎의질병_과습" )
                elif idx == 24:
                    test_eval.append( "잎의질병_과습여부")
                elif idx == 25:
                    test_eval.append( "잎의질병_마름"   )
                    beforeidx=idx
                elif idx == 26:
                    test_eval.append( "잎의질병_원인알수없음" )
                elif idx == 27:
                    test_eval.append(  "잎의질병_탄저병" )
                elif idx == 28:
                    test_eval.append( "해충_기타"   )
                    beforeidx=idx
                elif idx == 29:
                    test_eval.append(  "해충_깍지" )
                    beforeidx=idx                    
                elif idx == 30:
                    test_eval.append("해충_응애" )
                elif idx == 31:
                    test_eval.append("해충_해충약" )
                    beforeidx=idx

                
           
            print(">> 질문 카테고리: " + test_eval[0])
             
            answer = getAnswer(idx,0)
            print(">> 답변: "+answer)

        return answer,idx

 



In [ ]:
!pip install flask==0.12.2
!pip install flask-ngrok

In [ ]:
predict("깍지 때문에 고민이야",1)

>> 질문 카테고리: 해충_깍지
>> 답변: 깍지벌레 때문에 고민이시군요. 솜깍지벌레는 생장점위치에서 솜털 뭉치의 형태로 잘 발견되는 해충입니다. 깍지벌레는 스프레이형 살충제로는 잘 죽지 않아요. 다육이를 화분 채 약물에 30분~1시간 정도 담가주는 게 더 확실한 방제 방법입니다. 예방을 위해선 가끔씩 살충제를 가볍게 뿌려주시고, 통풍에 신경써주세요. 


('깍지벌레 때문에 고민이시군요. 솜깍지벌레는 생장점위치에서 솜털 뭉치의 형태로 잘 발견되는 해충입니다. 깍지벌레는 스프레이형 살충제로는 잘 죽지 않아요. 다육이를 화분 채 약물에 30분~1시간 정도 담가주는 게 더 확실한 방제 방법입니다. 예방을 위해선 가끔씩 살충제를 가볍게 뿌려주시고, 통풍에 신경써주세요. ',
 29)

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask import make_response
 
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run
#global beforeidx
#http://xxxxxx.ngrok.io/?msg=안녕
@app.route("/", methods=['GET','POST'])
def chatBot():
       global beforeidx 
       global s
       parameter_dict = request.args.to_dict()
       if len(parameter_dict) == 0:
           s="리피에게 궁금한걸 물어봐주세요~"
        
       else:
           question=request.args.get('msg')
           if question == '응':              
               s,tmp=predict(question,beforeidx)                
           elif question =='아니':               
               s,tmp=predict(question,beforeidx)                
           else:
               s,beforeidx =predict(question,-1)
       result = json.dumps({"answer": s}, ensure_ascii=False)      
       res = make_response(result)     
       return res, 200,{'content-type': 'application/json'}       
        
        
        
 

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

[Korean Sentence Splitter]:  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0713-35-224-108-158.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:18:17] "GET / HTTP/1.1" 200 -
[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:18:17] "GET /favicon.ico HTTP/1.1" 404 -
[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:18:18] "GET / HTTP/1.1" 200 -
[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:18:27] "GET /?msg=깍지%20때문에%20고민이야 HTTP/1.1" 200 -


>> 질문 카테고리: 해충_깍지
>> 답변: 깍지벌레 때문에 고민이시군요. 솜깍지벌레는 생장점위치에서 솜털 뭉치의 형태로 잘 발견되는 해충입니다. 깍지벌레는 스프레이형 살충제로는 잘 죽지 않아요. 다육이를 화분 채 약물에 30분~1시간 정도 담가주는 게 더 확실한 방제 방법입니다. 예방을 위해선 가끔씩 살충제를 가볍게 뿌려주시고, 통풍에 신경써주세요. 


[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:18:27] "GET /favicon.ico HTTP/1.1" 404 -
[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:18:34] "GET /?msg=해충약%20추천해줘 HTTP/1.1" 200 -


...

>> 질문 카테고리: 해충_응애
>> 답변: 응애 때문에 고민이시군요. 응애의 공격을 받은 다육이는 잎 뒤쪽에 후추 가루같은 점이 생깁니다. 응애는 옮을 수 있으므로 피해를 입은 다육이를 분리해주세요. 이후 2-3일 주기로 ‘응애 전용’ 살충제를 뿌려주세요. 


[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:19:47] "GET /?msg=분갈이할때%20화분은%20어떻게%20해? HTTP/1.1" 200 -


>> 질문 카테고리: 분갈이_화분
>> 답변: 다육이는 통풍과 배수가 중요하기 때문에 화분은 다리가 있어서 바닥 직접 닿지 않고 바닥에 구멍이 있는 화분이 좋아요. 화분 크기는 어떻게 키우느냐에 따라 달라집니다. 다육이를 번식시키고 싶으신가요?


[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:19:54] "GET /?msg=응 HTTP/1.1" 200 -
[Korean Sentence Splitter]: 127.0.0.1 - - [13/Nov/2021 10:20:02] "GET /?msg=다육이가%20아픈것같아 HTTP/1.1" 200 -


>> 질문 카테고리: 잎의질병_원인알수없음
>> 답변: 다육이가 아픈 것 같은데 원인을 몰라 답답하신가요? 진단 탭에서 사진을 찍어 지금 바로 진단 받아보세요! 다육이가 지금 어떤 병에 걸렸는지, 어떻게 대처해야 하는지 알려 드릴게요. 
